In [ ]:
import tkinter as tk
from tkinter import filedialog
import random
import math
import numpy as np

In [14]:
def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide the root window (we only want the file dialog)
    file_path = filedialog.askopenfilename(
        title="Select a file", 
        filetypes=[("Text files", "*.txt")]
    )
    return file_path

In [15]:
def parse_input(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Initialize variables
    vehicle_capacity = None
    customers = []

    # Extract vehicle capacity by looking for the "CAPACITY" line
    for i, line in enumerate(lines):
        if "CAPACITY" in line:
            # Get the next line for the actual values
            values_line = lines[i + 1].strip().split()
            vehicle_capacity = int(values_line[1])  # Extract the capacity value
            break

    if vehicle_capacity is None:
        raise ValueError("Vehicle capacity not found in the input file.")

    # Locate the start of the customer data
    start_index = None
    for i, line in enumerate(lines):
        if "CUST NO." in line:
            start_index = i + 1  # Data starts immediately after this line
            break

    if start_index is None:
        raise ValueError("Customer data section not found in file.")

    # Parse customer data
    for line in lines[start_index:]:
        data = line.strip().split()
        if len(data) < 7:  # Ignore lines without sufficient data
            continue
        try:
            customer = {
                "id": int(data[0]),
                "x": float(data[1]),
                "y": float(data[2]),
                "demand": int(data[3]),
                "ready_time": int(data[4]),
                "due_date": int(data[5]),
                "service_time": int(data[6])
            }
            customers.append(customer)
        except ValueError:
            # Skip malformed lines
            continue

    return vehicle_capacity, customers

In [17]:
# Function to calculate Euclidean distance between two customers
def euclidean_distance(cust1, cust2):
    return math.sqrt((cust1['x'] - cust2['x'])**2 + (cust1['y'] - cust2['y'])**2)

# Function to calculate travel time (using Euclidean distance as a proxy for time)
def calculate_travel_time(customers):
    """Create a time matrix based on travel distance."""
    num_customers = len(customers)
    time_matrix = [[0] * num_customers for _ in range(num_customers)]
    for i in range(num_customers):
        for j in range(num_customers):
            time_matrix[i][j] = euclidean_distance(customers[i], customers[j])
    return time_matrix

In [18]:
# Genetic Algorithm helpers
def generate_initial_population(population_size, num_customers):
    population = []
    for _ in range(population_size):
        individual = list(range(num_customers))
        random.shuffle(individual)
        population.append(individual)
    return population

In [19]:
def fitness(individual, dist_matrix, vehicle_capacity, customers):
    total_distance = 0
    total_demand = 0
    num_vehicles = 1  # Start with one vehicle
    
    for i in range(len(individual) - 1):
        current_customer = customers[individual[i]]
        next_customer = customers[individual[i + 1]]
        
        total_distance += dist_matrix[individual[i]][individual[i + 1]]
        total_demand += current_customer['demand']
        
        # Check if we exceed the vehicle's capacity
        if total_demand > vehicle_capacity:
            num_vehicles += 1
            total_demand = next_customer['demand']  # Start with the next customer's demand
            
    return total_distance  # You can modify to add penalty based on vehicles used

In [20]:
def crossover(parent1, parent2):
    # Perform crossover between two parents (partially mapped crossover)
    size = len(parent1)
    child = [-1] * size
    crossover_point = random.randint(0, size)
    
    # Copy part of the first parent to the child
    for i in range(crossover_point):
        child[i] = parent1[i]
    
    # Fill the remaining part with genes from the second parent
    for i in range(size):
        if child[i] == -1:
            for gene in parent2:
                if gene not in child:
                    child[i] = gene
                    break
    return child

In [24]:
import tkinter as tk
from tkinter import filedialog
import random
import math
import numpy as np

# Function to open file dialog and select a file
def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide the root window (we only want the file dialog)
    file_path = filedialog.askopenfilename(
        title="Select a file", 
        filetypes=[("Text files", "*.txt")]
    )
    return file_path

# Function to parse the input file and extract vehicle capacity and customer data
def parse_input(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Initialize variables
    vehicle_capacity = None
    customers = []

    # Extract vehicle capacity by looking for the "CAPACITY" line
    for i, line in enumerate(lines):
        if "CAPACITY" in line:
            # Get the next line for the actual values
            values_line = lines[i + 1].strip().split()
            vehicle_capacity = int(values_line[1])  # Extract the capacity value
            break

    if vehicle_capacity is None:
        raise ValueError("Vehicle capacity not found in the input file.")

    # Locate the start of the customer data
    start_index = None
    for i, line in enumerate(lines):
        if "CUST NO." in line:
            start_index = i + 1  # Data starts immediately after this line
            break

    if start_index is None:
        raise ValueError("Customer data section not found in file.")

    # Parse customer data
    for line in lines[start_index:]:
        data = line.strip().split()
        if len(data) < 7:  # Ignore lines without sufficient data
            continue
        try:
            customer = {
                "id": int(data[0]),
                "x": float(data[1]),
                "y": float(data[2]),
                "demand": int(data[3]),
                "ready_time": int(data[4]),
                "due_date": int(data[5]),
                "service_time": int(data[6])
            }
            customers.append(customer)
        except ValueError:
            # Skip malformed lines
            continue

    return vehicle_capacity, customers

# Function to calculate Euclidean distance between two customers
def euclidean_distance(cust1, cust2):
    return math.sqrt((cust1['x'] - cust2['x'])**2 + (cust1['y'] - cust2['y'])**2)

# Function to calculate travel time (using Euclidean distance as a proxy for time)
def calculate_travel_time(cust1, cust2):
    # Assuming travel time is proportional to the Euclidean distance (you can adjust this)
    return euclidean_distance(cust1, cust2)

# Genetic Algorithm helpers
def generate_initial_population(population_size, num_customers):
    population = []
    for _ in range(population_size):
        individual = list(range(num_customers))
        random.shuffle(individual)
        population.append(individual)
    return population

def fitness(individual, dist_matrix, vehicle_capacity, customers):
    total_distance = 0
    total_demand = 0
    num_vehicles = 1  # Start with one vehicle
    
    for i in range(len(individual) - 1):
        current_customer = customers[individual[i]]
        next_customer = customers[individual[i + 1]]
        
        total_distance += dist_matrix[individual[i]][individual[i + 1]]
        total_demand += current_customer['demand']
        
        # Check if we exceed the vehicle's capacity
        if total_demand > vehicle_capacity:
            num_vehicles += 1
            total_demand = next_customer['demand']  # Start with the next customer's demand
            
    return total_distance  # You can modify to add penalty based on vehicles used

def crossover(parent1, parent2):
    # Perform crossover between two parents (partially mapped crossover)
    size = len(parent1)
    child = [-1] * size
    crossover_point = random.randint(0, size)
    
    # Copy part of the first parent to the child
    for i in range(crossover_point):
        child[i] = parent1[i]
    
    # Fill the remaining part with genes from the second parent
    for i in range(size):
        if child[i] == -1:
            for gene in parent2:
                if gene not in child:
                    child[i] = gene
                    break
    return child

def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        # Randomly choose two positions to swap
        pos1, pos2 = random.sample(range(len(individual)), 2)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
    return individual

def select_parents(population, dist_matrix, vehicle_capacity, customers):
    # Tournament selection: select the best two individuals
    tournament_size = 5
    selected = random.sample(population, tournament_size)
    selected.sort(key=lambda x: fitness(x, dist_matrix, vehicle_capacity, customers))
    return selected[0], selected[1]

# Function to solve the VRP with Time Windows using Genetic Algorithm
def solve_vrptw(file_path):
    vehicle_capacity, customers = parse_input(file_path)
    num_customers = len(customers)
    
    # Create distance matrix
    dist_matrix = np.zeros((num_customers, num_customers))
    for i in range(num_customers):
        for j in range(num_customers):
            dist_matrix[i][j] = calculate_travel_time(customers[i], customers[j])
    
    # Genetic algorithm parameters
    population_size = 50
    generations = 100
    mutation_rate = 0.1
    
    # Initialize population
    population = generate_initial_population(population_size, num_customers)
    
    # Evolve the population
    for gen in range(generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = select_parents(population, dist_matrix, vehicle_capacity, customers)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            new_population.append(mutate(child1, mutation_rate))
            new_population.append(mutate(child2, mutation_rate))
        
        population = new_population
        
        # Track the best solution in this generation
        best_individual = min(population, key=lambda x: fitness(x, dist_matrix, vehicle_capacity, customers))
        print(f"Generation {gen + 1}, Best Fitness: {fitness(best_individual, dist_matrix, vehicle_capacity, customers)}")
    
    # Get the best individual from the final population
    best_individual = min(population, key=lambda x: fitness(x, dist_matrix, vehicle_capacity, customers))
    
    # Print optimized routes
    print("\nOptimized Routes:")
    routes = []
    total_distance = 0
    total_demand = 0
    num_vehicles = 1  # Start with one vehicle

    # Add the first customer as the starting point of the first vehicle's route
    routes.append(f"Vehicle {num_vehicles}: {best_individual[0]}")

    for i in range(1, len(best_individual)):
        current_customer = customers[best_individual[i - 1]]
        next_customer = customers[best_individual[i]]
        total_distance += dist_matrix[best_individual[i - 1]][best_individual[i]]
        total_demand += current_customer['demand']
        
        # Check if we exceed the vehicle's capacity
        if total_demand > vehicle_capacity:
            # If capacity is exceeded, start a new vehicle
            num_vehicles += 1
            total_demand = next_customer['demand']
            routes.append(f"Vehicle {num_vehicles}: {best_individual[i]}")
        else:
            # Otherwise, continue the route for the current vehicle
            routes[-1] += f" -> {best_individual[i]}"

    print("\n".join(routes))
    print(f"Total Distance: {total_distance}")
    print(f"Number of Vehicles Used: {num_vehicles}")

# Main function to select file and run the solver
file_path = select_file()

# Check if the user selected a file
if file_path:
    print(f"File selected: {file_path}")
    solve_vrptw(file_path)
else:
    print("No file selected. Exiting.")


File selected: C:/Users/chola/Documents/college/CESI/workshops/AA/PROJECT/c1/c101.txt
Generation 1, Best Fitness: 3431.577851965527
Generation 2, Best Fitness: 3404.603872879702
Generation 3, Best Fitness: 3404.603872879702
Generation 4, Best Fitness: 3433.1073337031567
Generation 5, Best Fitness: 3429.2171762687612
Generation 6, Best Fitness: 3394.0024852204165
Generation 7, Best Fitness: 3398.8573509839707
Generation 8, Best Fitness: 3364.766607593217
Generation 9, Best Fitness: 3278.069383310296
Generation 10, Best Fitness: 3298.8178171740356
Generation 11, Best Fitness: 3311.6259677978273
Generation 12, Best Fitness: 3244.9251779973656
Generation 13, Best Fitness: 3121.3724959678957
Generation 14, Best Fitness: 3141.2261692988186
Generation 15, Best Fitness: 3082.389352365697
Generation 16, Best Fitness: 3054.9891736649615
Generation 17, Best Fitness: 2950.2692598153426
Generation 18, Best Fitness: 2950.2692598153426
Generation 19, Best Fitness: 3003.4099376047707
Generation 20, Be